# Compute Penpals

In [1]:
import networkx as nx
import numpy as np
from scipy.spatial.distance import cdist
from lib.positions import coordArr

In [2]:
NR_xVal = 44 # the x-value at which the nerve ring segments the neurons
# determined by the x coordinate of RMEV, being 44.66 and posterior to the nerve ring
# TODO: use x and y coordinates to determine never ring location
# NOTE: Given thickness of nerve ring, x direction alone should be sufficient
# NOTE: New spatial data places RID on the posterior side of the nerve ring, may need to update how nerve ring is calculated as a result

def neuriteDirection(xPos):
  return 'tail' if (xPos < NR_xVal) else 'head'

# returns the list locations of the penpals for the head-facing neurons
# and the list locations of the penpals for the tail-facing neurons
def closestPenPals(L, R):
  distMatrix = cdist(L, R, 'sqeuclidean')
  return np.argmin(distMatrix, axis=1), np.argmin(distMatrix, axis=0)

# takes the ordered list of penpals and outputs a dictionary which labels the key as the first node and the value as the penpal
def labelPenPals(matchings, group1, group2):
  pals = {}
  for i,j in enumerate(matchings):
    label = group1[i]['label']
    label_penpal = group2[j]['label']
    pals[label] = label_penpal
  return pals

def findPenPals(G):
  penpals = {}
  segments = {'head':[],'tail':[]}
  coords = {'head':[],'tail':[]}

  for _,n in G.nodes(True):
    dir = neuriteDirection(n['x'])
    segments[dir].append(n)
    coords[dir].append(coordArr(n))

  penpals_head, penpals_tail = closestPenPals(coords['head'], coords['tail'])

  # merges the penpal dictionaries for both head and tail 
  penpals = labelPenPals(penpals_head, segments['head'], segments['tail']) |\
    labelPenPals(penpals_tail, segments['tail'], segments['head'])

  nx.set_node_attributes(G, penpals, 'PenPal')

In [3]:
def createPenPalGraph(folder):
  G = nx.read_gexf(f'{folder}/original/xyz.gexf')
  G_clean = nx.create_empty_copy(G)

  findPenPals(G_clean)
  nx.write_gexf(G_clean, f'{folder}/compiled/xyzp.gexf')

  for k,n in G_clean.nodes(True):
    G_clean.add_edge(k, n['PenPal'])
  nx.write_gexf(G_clean, f'{folder}/compiled/penpalSubgraph.gexf')

In [4]:
createPenPalGraph('data/kaiser/131')

In [4]:
createPenPalGraph('data/white/N2U')

In [5]:
for i in range(1,9):
  createPenPalGraph(f'data/witvliet/d{i}')